## reuters 
The Reuters Corpus contains 10,788 news documents totaling 1.3 million words. The documents have been classified into 90 topics, and grouped into two sets, called "training" and "test"; thus, the text with fileid 'test/14826' is a document drawn from the test set. This split is for training and testing algorithms that automatically detect the topic of a document, as we will see in chap-data-intensive.

In [1]:
from nltk.corpus import reuters
# len(reuters.fileids())
# reuters.fileids() #gets each file

#### calculating no. of grams

In [2]:
from collections import defaultdict
from nltk import ngrams
from funcy import flatten, isa

unigram=defaultdict(int)
bigram=defaultdict(int)
trigram=defaultdict(int)
fourgram=defaultdict(int)
fivegram=defaultdict(int)

def gram_calc(sentence,gram):
    if gram==1:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            unigram[gram]+=1
        return unigram
    elif gram==2:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            bigram[gram]+=1
        return bigram
    elif gram==3:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            trigram[gram]+=1
        return trigram
    elif gram==4:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            fourgram[gram]+=1
        return fourgram
    elif gram==5:
        temp=list(ngrams(sentence,gram))
        for gram in temp:
            fivegram[gram]+=1
        return fivegram

def start_stop_adder(news):
    sentence=[]
    temp=[]
    for word in news:
        # print(word)
        if word!=".":
            temp.append(word)
        elif word==" ":
            continue
        else:
            try:
                temp[0]="<s>"
                temp[len(temp)-1]="</s>"
            except IndexError:
                pass
            sentence.append([" ".join(temp)])
            temp=[]
    return sentence

for news in reuters.fileids():
    # print(news)
    sentences=start_stop_adder(reuters.words(news))
    para=[]
    for sentence in sentences:
        para.append(sentence[0].split(" "))
    para = list(flatten(para))
    gram_calc(para,1)
    gram_calc(para,2)
    gram_calc(para,3)
    gram_calc(para,4)
    gram_calc(para,5)
    

#### calculating probabilities

In [3]:
uni_count=0
for key,value in unigram.items():
    uni_count+=value
print(uni_count)

bi_count=0
for key,value in bigram.items():
    bi_count+=value
print(bi_count)

tri_count=0
for key,value in trigram.items():
    tri_count+=value
print(tri_count)

four_count=0
for key,value in fourgram.items():
    four_count+=value
print(four_count)

five_count=0
for key,value in fivegram.items():
    five_count+=value
print(five_count)

1569202
1559452
1549702
1539961
1530223


#### conditional probability : <br>
Bigram   : p(b|a)=p(a∩b)/p(a)<br>
trigram  : P(A| B,C) =P(A∩B∩C) /P(B∩C)<br>
fourgram : P(A| B,C,D) =P(A∩B∩C∩D) /P(B∩C∩D)<br>
fivegram : P(A| B,C,D,E) =P(A∩B∩C∩D∩E) /P(B∩C∩D∩E)<br>

In [4]:
uniprob=defaultdict(int)
for key, value in unigram.items():
    new_value = value/uni_count
    uniprob[key] = new_value

In [5]:
biprob=defaultdict(int)
for key, value in bigram.items():
    temp=[]
    temp.append(f'{key[1]}')
    temp=tuple(temp)
    prob_val=value/unigram[temp]
    biprob[key]=prob_val 
 

In [11]:
triprob=defaultdict(int)
for key, value in trigram.items():
    temp=key[1:3]
    prob_val=value/bigram[temp]
    triprob[key]=prob_val 

In [12]:
triprob

defaultdict(int,
            {('<s>', 'EXPORTERS', 'FEAR'): 1.0,
             ('EXPORTERS', 'FEAR', 'DAMAGE'): 1.0,
             ('FEAR', 'DAMAGE', 'FROM'): 0.5,
             ('DAMAGE', 'FROM', '</s>'): 0.03225806451612903,
             ('FROM', '</s>', '<s>'): 0.00021949078138718174,
             ('</s>', '<s>', '.-'): 1.0,
             ('<s>', '.-', 'JAPAN'): 1.0,
             ('.-', 'JAPAN', 'RIFT'): 1.0,
             ('JAPAN', 'RIFT', 'Mounting'): 1.0,
             ('RIFT', 'Mounting', 'trade'): 1.0,
             ('Mounting', 'trade', 'friction'): 0.06666666666666667,
             ('trade', 'friction', 'between'): 0.75,
             ('friction', 'between', 'the'): 0.016877637130801686,
             ('between', 'the', '</s>'): 0.005672793283412752,
             ('the', '</s>', '</s>'): 0.23589194971917626,
             ('</s>', '</s>', '<s>'): 0.08552393740639364,
             ('</s>', '<s>', 'Japan'): 1.0,
             ('<s>', 'Japan', 'has'): 0.23255813953488372,
             ('Ja

In [9]:
bigram

defaultdict(int,
            {('<s>', 'EXPORTERS'): 4,
             ('EXPORTERS', 'FEAR'): 1,
             ('FEAR', 'DAMAGE'): 1,
             ('DAMAGE', 'FROM'): 2,
             ('FROM', '</s>'): 31,
             ('</s>', '<s>'): 77452,
             ('<s>', '.-'): 150,
             ('.-', 'JAPAN'): 8,
             ('JAPAN', 'RIFT'): 1,
             ('RIFT', 'Mounting'): 1,
             ('Mounting', 'trade'): 1,
             ('trade', 'friction'): 15,
             ('friction', 'between'): 4,
             ('between', 'the'): 237,
             ('the', '</s>'): 4407,
             ('</s>', '</s>'): 7478,
             ('<s>', 'Japan'): 52,
             ('Japan', 'has'): 43,
             ('has', 'raised'): 23,
             ('raised', 'fears'): 4,
             ('fears', 'among'): 1,
             ('among', 'many'): 2,
             ('many', 'of'): 30,
             ('of', 'Asia'): 3,
             ('Asia', "'"): 4,
             ("'", 's'): 9133,
             ('s', 'exporting'): 1,
             ('


n = how many words has to be predicted<br>
grams=how many grams do user want us to predict the next word<br>



In [ ]:
#markov's assumption on chain rule of probability
